## Project: Machine Learning Classifier for Sorting Lego Pieces  
## Description:  
Developing and training algorithms for image-based classification of four different Lego categories in a conveyor system.

## Group#: 8  

## Authors:  
- Amer Alhamwi: 15976814  
- Wael Hamid: 80105687 

University of British Columbia Okanagan (UBCO)  

## Date:  
December 5, 2024  


In [148]:
# Import libraries essential for image processing, 
# data analysis, and machine learning.
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from PIL import Image, ImageFilter
from skimage import exposure
from scipy.ndimage import gaussian_filter,binary_dilation, binary_erosion
from numpy import asarray
from skimage.filters import threshold_otsu
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
from skimage.measure import find_contours
from sklearn.feature_selection import SequentialFeatureSelector



**Stage 1 code starts here:**

In [149]:
# The following code loads images from the specified folder, applies image cropping 
# to enhance training accuracy, and assigns classification labels based on the first 
# letter of each image's name. The processed images and labels are stored in arrays 
# for subsequent use in the training process.


# Declare the two folders names for image files
path = "c:/Users/Admin/Desktop/ENGR 418/Assignments/Project/Project_Stage_2/"

def test_function(path):
    folder_training = os.path.join(path, "training")
    folder_testing = os.path.join(path, "testing")
    
    return folder_training, folder_testing

folder_training, folder_testing = test_function(path)

# Define the function before calling it
def get_image_data(folder):
    x = []
    y = []

    for pic in os.listdir(f"{folder}/"):
        # opening image using image from PIL
        image = Image.open(f"{folder}/{pic}").convert("L")
        # Crop the images to the center to make it more accurate
        top = int(np.floor(image.height / 4))
        bottom = int(np.ceil(image.height * (3 / 4)))
        left = int(np.floor(image.width / 4))
        right = int(np.ceil(image.width * (3 / 4)))
        #print(image.size)

        # new image is about [500*500] pixels
        cropped_image = image.crop((left, top, right, bottom))

        # resizing images to 64*64 pixels (4096)
        resized_image = cropped_image.resize((64, 64))
        
        data = np.asarray(resized_image)  # Convert image to an array
        
        vec = np.hstack(data)  # Flatten 2D image to a 1D array
        
        x.append(vec)  # Append the flattened image data to the list

        # Assign classes based on first letter/number of the image name
        if str.lower(pic[0]) == "c":
            y.append(0)  # Cir
        elif str.lower(pic[0]) == "r":
            y.append(1)  # rec
        elif str.lower(pic[0]) == "2":
            y.append(2)  # 2b1
        else:
            y.append(3)  # Squ

    x = np.array(x)  # Convert to numpy array
    y = np.array(y)  # Convert to numpy array

    #print(x)
    #print(y)
    return x, y



In [150]:
# This cell loads the training data, trains a Logistic 
# Regression model with the training set, and then
# evaluates the model by predicting on the same training data.

#Set paths for training and testing folders based on the project path
folder_training, folder_testing = test_function(path)

x1_train, y1_train = get_image_data(folder_training)

# Train the Logistic Regression model 
model = LogisticRegression(solver='lbfgs', max_iter=1000)  # Increase max_iter to 1000 or more

#print(f"Data from first directory: {x1_train.shape}, Labels: {y1_train.shape}")

# Train the Logistic Regression model 
model = LogisticRegression(solver='lbfgs', max_iter=1000)  # Increase max_iter to 1000 
model.fit(x1_train, y1_train)
y_pred1 = model.predict(x1_train)

print("Accuracy:", np.round(accuracy_score(y1_train, y_pred1),3))
print("Confusion Matrix:\n", confusion_matrix(y1_train, y_pred1))

Accuracy: 1.0
Confusion Matrix:
 [[27  0  0  0]
 [ 0 27  0  0]
 [ 0  0 27  0]
 [ 0  0  0 27]]


In [151]:
# This code block loads the testing data, uses the trained Logistic Regression model 
# to make predictions on the test set, and evaluates the model's performance by 
# displaying the accuracy score and a confusion matrix.

# Load the testing data
x1_test, y1_test = get_image_data(folder_testing)

# Predict on the test set
y_pred2 = model.predict(x1_test)

# Print the accuracy and confusion matrix
print("Accuracy:", np.round(accuracy_score(y1_test, y_pred2), 3))
print("Confusion Matrix:\n", confusion_matrix(y1_test, y_pred2))

Accuracy: 0.454
Confusion Matrix:
 [[15  3  5  4]
 [ 3 16  1  7]
 [ 9  4  8  6]
 [10  6  1 10]]


**Stage 2 code starts here:**

In [152]:
# This cell sets the project folder path and includes two function definitions:  
# test_function identifies the training and testing directories, and  
# get_image_data processes images into flattened arrays along with their labels.

#test_function finds the training and testing folders 
def test_function(path):
    folder_training = os.path.join(path, "training/")
    folder_testing = os.path.join(path, "testing/")
    
    return folder_training, folder_testing

folder_training, folder_testing = test_function(path)

#get_image_data returns arrays to store the image horizontallty and their labels 
def get_image_data(folder, im_width, im_length):
    
    file_names = os.listdir(folder)
    n_samples = len(file_names)
    
    #Decalre arrays to store the image horizontallty and their labels 
    x = np.empty((n_samples, im_width * im_length))
    y = np.empty((n_samples, 1), dtype=int)
    #declaring incrementer to go throght the files 
    i=0

    for pic in os.listdir(f"{folder}/"):
        
        #opening image using image from PIL
        image = Image.open(f"{folder}/{pic}").convert("L")
        #resizing image
        resized_image = image.resize((im_width, im_length))
        
        #Convert image to array and store it hoirizonatallty(row)
        image_array = asarray(resized_image)
        x[i, :] = image_array.reshape(1, -1)
        
        #Assign classes based on first letter/number of the image file name
        if str.lower(pic[0]) == "c":
            y[i] = 0  # Cir
        elif str.lower(pic[0]) == "r":
            y[i] = 1  # Rec
        elif str.lower(pic[0]) == "2":
            y[i] = 2  # 2b1
        else:
            y[i] = 3  # Squ
        i+=1        
    return x, y.ravel()  

In [153]:
# This cell processes images using edge detection techniques to extract features.  
# It includes edge filtering, thresholding, and line detection to calculate key metrics.  
# The process_image function returns a feature vector to train the model effectively.

#process_image returns the features that will be passed to train the model 
def process_image(image_array,y_train,ET):

    #decalring image dimensions 
    im_width=64
    im_length=64

    #horizontal/vertical lines & their count values
    horizontal_line = np.zeros(im_width)
    vertial_line = np.zeros(im_length)
    h_count=0
    v_count=0
    
    #degree incrementer
    deg_inc=0
    # Edge density accumulator
    edge_density = 0

    #First,start by applying image proccessing and edge filtering... 
    global_mean = np.mean(image_array)
    image_array = image_array - global_mean  # Subtract the global mean
    # increase contrast by making image occupy the whole grayscale 0 to 255
    image_array = (image_array-np.min(image_array))*255/(np.max(image_array)-np.min(image_array))
    # reshpae image to 64x64 and convert to grayscale
    im = Image.fromarray(image_array.reshape(im_width,im_length)).convert('L') 
    # detect the edges and keep them
    edges_image = im.filter(ImageFilter.FIND_EDGES) 
    #convert the edges to array type 
    edges_array = np.asarray(edges_image).astype(float) 
    # edge filter produces artificial edges at the boundary, we remove them next
    edges_array_scaled = edges_array.copy()[1:im_width-3,1:im_length-3]
    # apply edge thresholding and keep only prominent ones
    edges_array_scaled[edges_array_scaled < ET,] = 0
    #create an image from our final array
    im_rotate = Image.fromarray(edges_array_scaled)      

    # Next, find horizontal lines and vertical lines in the image 

    #the goal is keep the biggest value of horzintal and vertical count
    while(deg_inc<=360):
        # Rotate the PIL Image
        rotated_image = im_rotate.rotate(deg_inc)
        # Convert rotated image back to a NumPy array
        rotated_array = np.asarray(rotated_image)
        for y in range(im_width-4):
            horizontal_line[y] = np.count_nonzero(rotated_array[y, :]) 
            if y > 0 and horizontal_line[y] >= horizontal_line[y-1]:
                h_count = horizontal_line[y]  # Save the biggest horizontal count

        for x in range(im_length-4):
            vertial_line[x] = np.count_nonzero(rotated_array[:,x])                         
            if x > 0 and vertial_line[x] > vertial_line[x-1]:
                v_count = vertial_line[x]     # Save the biggest vertical count

  
        # Compute edge density for the current rotation
        edge_density += (np.count_nonzero(rotated_array) / (rotated_array.size))
        deg_inc= deg_inc + 1   

    # plt.imshow(rotated_image)
    # plt.show()

    # Compute sum of vertical and horizontal pixel intensities
    edges_v = np.sum(rotated_image,axis=0)
    edges_h = np.sum(rotated_image,axis=1)

    # Compute 45-degree diagonal edges pixel intensities
    edges_diag_45 = [np.sum(np.diag(np.fliplr(edges_array_scaled), k)) for k in range(-edges_array_scaled.shape[0] + 1, edges_array_scaled.shape[1])]
    # Compute -45-degree diagonal edges  pixel intensities
    edges_diag_neg45 = [np.sum(np.diag(edges_array_scaled, k)) for k in range(-edges_array_scaled.shape[0] + 1, edges_array_scaled.shape[1])]


    #This definition returns the first and last non zero values in a rotated image
    #used to calculate and calc the distance between them
    def get_nonzero_segments(rotated_image):

        # Get the indices of all non-zero elements in the array
        nonzero_indices = np.nonzero(rotated_image)[0]

        # If there are at least 6 non-zero elements
        if len(nonzero_indices) >= 6:
            # Extract the first non-zero index
            first_nonzero = nonzero_indices[:1]
            # Extract the last non-zero index
            last_nonzero = nonzero_indices[-1:]
        else:  # Cases with fewer than 6 non-zero elements
            # Extract all non-zero indices as the first segment
            first_nonzero = nonzero_indices[:len(nonzero_indices) // 1]
            # Extract all non-zero indices as the last segment
            last_nonzero = nonzero_indices[len(nonzero_indices) // 1:]
        return first_nonzero, last_nonzero

    #This definition returns the first and last non zero values in an image 
    #used to calculate and calc the distance between them
    def get_nonzero_segments1(edge_array):

        # Get the indices of all non-zero elements in the array
        nonzero_indices = np.nonzero(edge_array)[0]

        # If there are at least 6 non-zero elements
        if len(nonzero_indices) >= 6:
            # Extract the first non-zero index
            first_nonzero = nonzero_indices[:1]
            # Extract the last non-zero index
            last_nonzero = nonzero_indices[-1:]
        else:  # Cases with fewer than 6 non-zero elements
            # Extract all non-zero indices as the first segment
            first_nonzero = nonzero_indices[:len(nonzero_indices) // 1]
            # Extract all non-zero indices as the last segment
            last_nonzero = nonzero_indices[len(nonzero_indices) // 1:]
        return first_nonzero, last_nonzero
       
    # Find first and last non zero value for each vector     
    first_v, last_v = get_nonzero_segments(edges_v)
    first_h, last_h = get_nonzero_segments(edges_h) 
    first_45, last_45 = get_nonzero_segments1(edges_diag_45)
    first_neg45, last_neg45 = get_nonzero_segments1(edges_diag_neg45)

    #This commmented section is used to display our results and make sense of our features
    ######################################################################################
    # print(last_v)
    # print(first_v)
    # print(v_distance)
    # plt.plot(first_v, edges_v[first_v],last_v,edges_v[last_v], label="Vertical Edges", color='blue')
    # plt.plot(first_h, edges_h[first_h],last_h,edges_h[last_h], label="Horizontal  Edges", color='orange')
    # plt.legend(loc="upper right")  # You can adjust loc to place the legend
    ######################################################################################

    # Find the distance for each vector  
    v_distance = last_v - first_v
    h_distance = last_h - first_v
    distance_45 = last_45 - first_45
    distance_neg45 = last_neg45 - first_neg45

    # Compute the horizontal/vertical ratio feature
    HV_ratio = h_distance*v_distance

    #This commmented section is used to monnitor all distanes and make sense of our features
    ########################################################################################
    # print(last_v)
    # print(first_v)
    # print(v_distance)
    # plt.plot(first_v, edges_v[first_v],last_v,edges_v[last_v], label="Vertical Edges", color='blue')
    # plt.plot(first_h, edges_h[first_h],last_h,edges_h[last_h], label="Horizontal  Edges", color='orange')
    # plt.legend(loc="upper right")  # You can adjust loc to place the legend
    ########################################################################################

    # Find the highest vertical and horizontal peaks 
    peaks_v, _ = find_peaks(edges_v)
    peaks_h, _ = find_peaks(edges_h)


    # Store the largest vertical and horzitonatl peaks and normalize them
    largest_peak_v_idx = peaks_v[np.argmax(edges_v[peaks_v])]  # Index of the largest peak
    largest_peak_v_value = (edges_v[largest_peak_v_idx]/255)   # Value of the largest peak
    largest_peak_h_idx = peaks_h[np.argmax(edges_h[peaks_h])]  # Correctly use peaks_h
    largest_peak_h_value = (edges_h[largest_peak_h_idx]/255)   # Value of the largest peak

    # Ensure scalar values for all features
    first_v = first_v.item() 
    v_distance = v_distance.item() 
    first_h = first_h.item() 
    h_distance = h_distance.item() 
    HV_ratio = HV_ratio.item() 
    first_45 = first_45.item() 
    distance_45 = distance_45.item() 
    first_neg45 = first_neg45.item() 
    distance_neg45 = distance_neg45.item() 
    
    # Create feature vector with the 13 engineerd features and flatten 
    x = np.array([h_count,edge_density,first_v,v_distance ,first_h, h_distance, HV_ratio,first_45,distance_45 ,first_neg45,distance_neg45,largest_peak_v_value,largest_peak_h_value]).reshape(1, -1)
    #print(x)
    return x # return the 13 features


In [154]:
# This cell loads the training data, trains a Logistic Regression model
# to make predictions on the taining set, and evaluates the model's performance by 
# displaying the accuracy score and a confusion matrix.

#Decalring the edge threshold & image dimensions 
ET = 100
width= 64
length= 64

#Declaring training arrays
file_names_train = os.listdir(folder_training)
length_train = len(file_names_train)
x_train = np.empty((length_train, width)) 
y_train = np.empty((length_train, 1))

# Load the training data
x_train, y_train = get_image_data(folder_training, width, length)

# Load the 13 engineered features to use to train our model
feature_train = np.empty((length_train, 13))
for i in range(length_train):
    feature_train[i, :] = process_image(x_train[i, :],y_train,ET)
 

#Train the Logistic Regression model 
model = LogisticRegression(max_iter=5000)
model.fit(feature_train, y_train)

# Use Sequential Feature Selector to bring down the # of features
# Note: this was used when we had more than 13 features
# sfs = SequentialFeatureSelector(model, n_features_to_select=13)
# sfs.fit(featur_train, y_train)
# print(sfs.get_support())

#Display Training Accuracy & Confusion Matrix 
y_train_pred = model.predict(feature_train)
print("Accuracy:", np.round(accuracy_score(y_train, y_train_pred),3))
print("Confusion Matrix:\n",confusion_matrix(y_train, y_train_pred))

Accuracy: 0.917
Confusion Matrix:
 [[21  0  6  0]
 [ 0 27  0  0]
 [ 3  0 24  0]
 [ 0  0  0 27]]


In [155]:
# This cell loads the testing data, uses the trained Logistic Regression model 
# to make predictions on the test set, and evaluates the model's performance by 
# displaying the accuracy score and a confusion matrix.

#Declaring test arrays
file_names_test = os.listdir(folder_testing)
length_test = len(file_names_test)
x_test = np.empty((length_test, width))
y_test = np.empty((length_test, 1))

# Load the testing data
x_test, y_test = get_image_data(folder_testing, width, length)

#load the 13 engineered features to use to test our model
feature_test = np.empty((length_test, 13))
for i in range(length_test):
    feature_test[i, :] = process_image(x_test[i, :],y_test[i],ET)

# Display Testing Accuracy & Confusion Matrix 
y_test_pred = model.predict(feature_test)
print("Accuracy:", np.round(accuracy_score(y_test, y_test_pred),3))
print("Confusion Matrix:\n",confusion_matrix(y_test, y_test_pred))  

Accuracy: 0.898
Confusion Matrix:
 [[21  0  6  0]
 [ 0 27  0  0]
 [ 1  0 23  3]
 [ 0  0  1 26]]
